In [1]:
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
#Lets use chardet to detect the type of encoding used in this file
import chardet
chardet.detect(open('data/world_bank_projects.json','rb').read())#confidence 1 implies that the file is 100% ascii

{'confidence': 1.0, 'encoding': 'ascii', 'language': ''}

In [3]:
data = pd.read_json('data/world_bank_projects.json',encoding='ascii')
data.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [4]:
data.shape #500 rows by 50 columns

(500, 50)

In [5]:
data.info()#sanity check for null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

#### * 10 countries with most projects

In [6]:
#10 countries with most projects
data.countryname.value_counts().head(10)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

####  * 10 major project themes

In [7]:
#10 major project themes

#load json file as string
import json
str_data=json.load(open('data/world_bank_projects.json'))
project_themes = json_normalize(str_data,record_path='mjtheme_namecode',meta='project_name')

#set index to code
project_themes = project_themes.set_index('code')
project_themes.head()

,name,project_name
code,,
8,Human development,Ethiopia General Education Quality Improvement...
11,,Ethiopia General Education Quality Improvement...
1,Economic management,TN: DTF Social Protection Reforms Support
6,Social protection and risk management,TN: DTF Social Protection Reforms Support
5,Trade and integration,Tuvalu Aviation Investment Project - Additiona...


In [8]:
project_themes.name.value_counts().head(10) #10 major project themes

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

#### * Fill the missing name fields

In [9]:
#Fill the missing name fields

#load json file as string
import json
str_data=json.load(open('data/world_bank_projects.json'))
project_themes = json_normalize(str_data,record_path='mjtheme_namecode',meta='project_name')

#convert code from str to int
project_themes['code'] = project_themes['code'].map(lambda x: int(x))

#set index to code
project_themes = project_themes.set_index('code')
project_themes.tail()

,name,project_name
code,,
10,Rural development,Sustainable Management of Agricultural Researc...
9,Urban development,KENYA: NATIONAL URBAN TRANSPORT IMPROVEMENT PR...
8,Human development,KENYA: NATIONAL URBAN TRANSPORT IMPROVEMENT PR...
5,Trade and integration,KENYA: NATIONAL URBAN TRANSPORT IMPROVEMENT PR...
4,Financial and private sector development,KENYA: NATIONAL URBAN TRANSPORT IMPROVEMENT PR...


In [10]:
#Each code has its name
#Lets make a dictionay of codes and names
#Because ffill or bfill may fill the name with wrong name 
#Example:
#    Code     name
#    2        A
#    3        ''
#    3        B
#    3        ''
#    4        C
#This results in a wrong name 'A' when we use ffill and bfill results in 'C' which is also wrong

import numpy                          # Because .unique returns an ndarray of 1 dimension
names = numpy.delete(project_themes.name.sort_index().unique(),[0]) #Delete 2nd element it is ''
codes_dict = {}
for i,name in enumerate(names,start=1):
    codes_dict[i] = name
codes_dict

{1: 'Economic management',
 2: 'Public sector governance',
 3: 'Rule of law',
 4: 'Financial and private sector development',
 5: 'Trade and integration',
 6: 'Social protection and risk management',
 7: 'Social dev/gender/inclusion',
 8: 'Human development',
 9: 'Urban development',
 10: 'Rural development',
 11: 'Environment and natural resources management'}

In [11]:
#Lets write a function to take a Dataframe and replace null values in name column with name specific to its code
def replace_name(df,code_dict):
    df = df.reset_index()
    for key,val in code_dict.items():
        df_temp = df[(df.code==key) & (df.name=='')]     
        df.loc[df_temp.index.tolist(), 'name']=val
    df = df.set_index('code')
    return df

In [12]:
project_themes = replace_name(project_themes,codes_dict)
len(project_themes[project_themes.name=='']) #So, no empty name fields if 0 is length

0

In [13]:
project_themes  #previous '' disappeared

,name,project_name
code,,
8,Human development,Ethiopia General Education Quality Improvement...
11,Environment and natural resources management,Ethiopia General Education Quality Improvement...
1,Economic management,TN: DTF Social Protection Reforms Support
6,Social protection and risk management,TN: DTF Social Protection Reforms Support
5,Trade and integration,Tuvalu Aviation Investment Project - Additiona...
2,Public sector governance,Tuvalu Aviation Investment Project - Additiona...
11,Environment and natural resources management,Tuvalu Aviation Investment Project - Additiona...
6,Social protection and risk management,Tuvalu Aviation Investment Project - Additiona...
7,Social dev/gender/inclusion,Gov't and Civil Society Organization Partnership
